In [1]:
#pip install tf-nightly
#pip install tfp-nightly

pseudobulks

In [16]:
import os 
os.getcwd()

'/home/daveg/Work/cruk/pseudobulks'

In [2]:
#%reset

In [3]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import numpy as np
import anndata
import itertools
import gc
#from diffexpr.py_deseq import py_DESeq2
#from rpy2.robjects import Formula

In [5]:
q = sc.read_h5ad('../atlas/Atlas_adatas_June2021_Atlas_final_May2021.h5ad')

In [6]:
def build_design(q, qci):
    # build design matrix
    patient_ids = ([x[0:3] for x in qci.obs.samplename])
    full_sample_df = pd.DataFrame({'patient':patient_ids, 'biosample':qci.obs.samplename, 'dx':qci.obs.diagnosis})
    # get the number of cells from each sample
    cell_counts = pd.DataFrame(full_sample_df.biosample.value_counts())
    cell_counts.columns = ['cell_counts']
    cell_counts['biosample'] = cell_counts.index
    # merge in the cell counts
    full_sample_df = full_sample_df.merge(cell_counts)
    # the list of biosamples in this cluster
    biosample_list = list(set(full_sample_df.biosample))
    # and the order of cells as index
    index = np.array(full_sample_df.biosample.tolist())
    # then we make the design matrix
    sample_df = full_sample_df.drop_duplicates()
    sample_df.loc[:,'binned_cell_counts'] = pd.cut(sample_df.cell_counts, bins=[0,5,10,20,40,80,160,100000]) #((sample_df.cell_counts - np.mean(sample_df.cell_counts)) / np.std(sample_df.cell_counts))
    return( (biosample_list, index, sample_df) )

In [7]:
def build_count_matrix(biosample_list, index, qci, sample_df):
    # sum within samples
    res0 = pd.DataFrame()
    for bsl in biosample_list:
        idx = np.argwhere(index == bsl).flatten()
        mat = qci.X[idx,:].sum(axis=0)
        cnt_sum = mat.flatten().tolist()[0]
        if len(res0) == 0:
            res0 = pd.DataFrame(cnt_sum, columns=[bsl])
        else:
            res0 = res0.join(pd.DataFrame(cnt_sum, columns=[bsl]))
    count_matrix = res0.loc[:, sample_df.biosample.tolist()]
    count_matrix['id'] = qci.var.index.tolist()
    count_matrix.index = qci.var.index
    return(count_matrix)

In [10]:
# batch
cellclusters = dict(
    epithelial=['0','3','4','6','8','13','19','20','28','31','33','34','35','36','38','39','40'], # 78049 cells, # 32 is heptoid
    squamous=['16','18','21'],
    fibroblasts=['7'],
    myofibroblasts=['15'],
    endothelial=['12','15','5', '30', '26'],
    stromal=['7','12','15','5', '30', '26'],
    neutrophils=['22'],
    Bcells=['11','23'],  # 23 plasma?
    monocytes=['10'],  # and macs and dcs
    cd4_Tcells=['2'],
    cd8_Tcells=['1','25'],
    NKcells=['14'],
    mastcells=['9'],
    parietal_cell=['42'],
    chief_cell=['4','6']
)

In [9]:
# non batch
cellclusters = dict(
    epithelial=['0','3','4','6','8','13','19','20','28','31','33','34','35','36','38','39','40'], # 78049 cells, # 32 is heptoid
    squamous=['16','18','21'],
    fibroblasts=['7'],
    myofibroblasts=['15'],
    endothelial=['12','15','5', '30', '26'],
    stromal=['7','12','15','5', '30', '26'],
    neutrophils=['22'],
    Bcells=['11','23'],  # 23 plasma?
    monocytes=['10'],  # and macs and dcs
    cd4_Tcells=['2'],
    cd8_Tcells=['1','25'],
    NKcells=['14'],
    mastcells=['9'],
    parietal_cell=['42'],
    chief_cell=['4','6']
)

In [11]:
for leiden_label in cellclusters.keys():
    print(leiden_label)

epithelial
squamous
fibroblasts
myofibroblasts
endothelial
stromal
neutrophils
Bcells
monocytes
cd4_Tcells
cd8_Tcells
NKcells
mastcells
parietal_cell
chief_cell


In [13]:
clusterlabs = list(set(q.obs.leiden))
clusterlabs.sort()
res_df = pd.DataFrame()
for leiden_label in cellclusters.keys(): #clusterlabs:
    # subset the anndata to this cluster
    print('leiden cluster: ' + leiden_label)
    ### subset data to this cluster
    clusterlabels = cellclusters[leiden_label]
    qci = q[q.obs.leiden.isin(clusterlabels)]
    #qci = qci[qci.obs.diagnosis.isin(['NE', 'NS', 'M', 'D', 'T'])] # use all samples
    ###
    (biosample_list, index, sample_df) = build_design(q, qci)
    # building the pseudobulk count matrix
    count_matrix = build_count_matrix(biosample_list, index, qci, sample_df)
    print('columns in count matrix: ' + str(len(count_matrix.columns)))
    sample_df.binned_cell_counts = [str(x) for x in sample_df.binned_cell_counts]
    #write it out
    count_matrix.to_csv('pseudobulks_cluster_level/'+leiden_label+'_pseudobulk.csv')
    sample_df.to_csv('pseudobulks_cluster_level/'+leiden_label+'_metadata.csv')    
    del qci
    gc.collect()


leiden cluster: epithelial


/home/daveg/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


columns in count matrix: 57
leiden cluster: squamous
columns in count matrix: 46
leiden cluster: fibroblasts
columns in count matrix: 57
leiden cluster: myofibroblasts
columns in count matrix: 54
leiden cluster: endothelial
columns in count matrix: 57
leiden cluster: stromal
columns in count matrix: 57
leiden cluster: neutrophils
columns in count matrix: 45
leiden cluster: Bcells
columns in count matrix: 56
leiden cluster: monocytes
columns in count matrix: 54
leiden cluster: cd4_Tcells
columns in count matrix: 57
leiden cluster: cd8_Tcells
columns in count matrix: 57
leiden cluster: NKcells
columns in count matrix: 57
leiden cluster: mastcells
columns in count matrix: 57
leiden cluster: parietal_cell
columns in count matrix: 1
leiden cluster: chief_cell
columns in count matrix: 55


In [14]:
cellnames = list(set(q.obs.CL_name))
cellnames

['pancreatic epsilon cell',
 'lung secretory cell',
 'enterocyte',
 'T cell',
 'lymphocyte',
 'antibody secreting cell',
 'epithelial cell of stomach',
 'neutrophil',
 'striated muscle cell',
 'peritubular myoid cell',
 'unknown',
 'ciliated columnar cell of tracheobronchial tree',
 'pancreatic A cell',
 'fibroblast',
 'dendritic cell',
 'hepatocyte',
 'mast cell',
 'myofibroblast cell',
 'parietal cell',
 'smooth muscle cell',
 'epithelial cell of lung',
 'Schwann cell',
 'endothelial cell',
 'peptic cell',
 'mucus neck cell of gastric gland',
 'neuron',
 'pulmonary ionocyte',
 'pancreatic stellate cell',
 'pancreatic PP cell',
 'hematopoietic precursor cell',
 'natural killer cell',
 'pancreatic ductal cell',
 'granulocyte',
 'luminal epithelial cell of mammary gland',
 'goblet cell',
 'respiratory basal cell',
 'B cell',
 'epithelial cell of esophagus',
 'macrophage',
 'chromaffin cell',
 'mucous cell of stomach',
 'pancreatic D cell',
 'plasmacytoid dendritic cell']

In [17]:

def buildPM(qci,q,cell_label):
    #qci = qci[qci.obs.diagnosis.isin(['NE', 'NS', 'M', 'D', 'T'])]
    ###
    (biosample_list, index, sample_df) = build_design(q, qci)
    # building the pseudobulk count matrix
    count_matrix = build_count_matrix(biosample_list, index, qci, sample_df)
    sample_df.binned_cell_counts = [str(x) for x in sample_df.binned_cell_counts]
    #write it out
    count_matrix.to_csv('pseudobulks_celltype_level/'+cell_label+'_pseudobulk.csv')
    sample_df.to_csv('pseudobulks_celltype_level/'+cell_label+'_metadata.csv')


res_df = pd.DataFrame()
for cell_label in cellnames: #clusterlabs:
    # subset the anndata to this cluster
    print('celltype: ' + cell_label)
    if cell_label == 'neutrophil':
        ### subset data to this cluster
        print('***monocytes***')
        qci = q[(q.obs.CL_name == cell_label) & (q.obs.leiden == '10')]
        buildPM(qci,q,'monocytes')
        print('***neutrophils***')
        qci = q[(q.obs.CL_name == cell_label) & (q.obs.leiden == '22')]
        buildPM(qci,q,'neutrophils')
    else:
        ### subset data to this cluster
        qci = q[q.obs.CL_name == cell_label]
        buildPM(qci,q,cell_label)
        
    del qci
    gc.collect()


celltype: pancreatic epsilon cell
celltype: lung secretory cell
celltype: enterocyte
celltype: T cell
celltype: lymphocyte
celltype: antibody secreting cell
celltype: epithelial cell of stomach
celltype: neutrophil
***monocytes***
***neutrophils***
celltype: striated muscle cell
celltype: peritubular myoid cell
celltype: unknown
celltype: ciliated columnar cell of tracheobronchial tree
celltype: pancreatic A cell
celltype: fibroblast
celltype: dendritic cell
celltype: hepatocyte
celltype: mast cell
celltype: myofibroblast cell
celltype: parietal cell
celltype: smooth muscle cell
celltype: epithelial cell of lung
celltype: Schwann cell
celltype: endothelial cell
celltype: peptic cell
celltype: mucus neck cell of gastric gland
celltype: neuron
celltype: pulmonary ionocyte
celltype: pancreatic stellate cell
celltype: pancreatic PP cell
celltype: hematopoietic precursor cell
celltype: natural killer cell
celltype: pancreatic ductal cell
celltype: granulocyte
celltype: luminal epithelial ce

In [8]:
samplenames = list(set(q.obs.samplename))
samplenames

['E07A',
 'E21B',
 'E20D',
 'E19F',
 'E17C',
 'E26B',
 'E26D',
 'E24B',
 'E23A',
 'E24D',
 'E24A',
 'E19A',
 'E07D1',
 'E12C',
 'E08C',
 'E08B',
 'E08D',
 'E12D',
 'E07B',
 'E21F',
 'E14B',
 'E19E',
 'E11D',
 'E11C',
 'E19B',
 'E21E',
 'E12B',
 'E17B',
 'E24C',
 'E14C',
 'E17A',
 'E17E',
 'E23E',
 'E11E',
 'E14D',
 'E21C',
 'E20A',
 'E26E',
 'E26C',
 'E26A',
 'E07D2',
 'E20B',
 'E23B',
 'E19D',
 'E21A',
 'E23C',
 'E23D',
 'E17D',
 'E19C',
 'E21D',
 'E20C',
 'E11B',
 'E14A',
 'E08A',
 'E07C',
 'E12A']

In [11]:


res_df = pd.DataFrame()

for sample_label in samplenames: #clusterlabs:
    # subset the anndata to this cluster
    print('sample name: ' + sample_label)
    ### subset data to this cluster
    qci = q[q.obs.samplename == sample_label]
    mat = qci.X.sum(axis=0)
    cnt_sum = mat.flatten().tolist()[0]
    if len(res_df) == 0:
        res_df = pd.DataFrame(cnt_sum, columns=[sample_label])
    else:
        res_df = res_df.join(pd.DataFrame(cnt_sum, columns=[sample_label]))
    del qci
    gc.collect()

res_df.to_csv('atlas_samplenames_all_genes_pseudobulk.csv')


sample name: E07A
sample name: E21B
sample name: E20D
sample name: E19F
sample name: E17C
sample name: E26B
sample name: E26D
sample name: E24B
sample name: E23A
sample name: E24D
sample name: E24A
sample name: E19A
sample name: E07D1
sample name: E12C
sample name: E08C
sample name: E08B
sample name: E08D
sample name: E12D
sample name: E07B
sample name: E21F
sample name: E14B
sample name: E19E
sample name: E11D
sample name: E11C
sample name: E19B
sample name: E21E
sample name: E12B
sample name: E17B
sample name: E24C
sample name: E14C
sample name: E17A
sample name: E17E
sample name: E23E
sample name: E11E
sample name: E14D
sample name: E21C
sample name: E20A
sample name: E26E
sample name: E26C
sample name: E26A
sample name: E07D2
sample name: E20B
sample name: E23B
sample name: E19D
sample name: E21A
sample name: E23C
sample name: E23D
sample name: E17D
sample name: E19C
sample name: E21D
sample name: E20C
sample name: E11B
sample name: E14A
sample name: E08A
sample name: E07C
sample n

In [13]:
res_df

,E07A,E21B,E20D,E19F,E17C,E26B,E26D,E24B,E23A,E24D,...,E23D,E17D,E19C,E21D,E20C,E11B,E14A,E08A,E07C,E12A
0,428.0,1072.0,361.0,2491.0,319.0,1306.0,2378.0,815.0,657.0,742.0,...,402.0,390.0,911.0,404.0,354.0,215.0,599.0,130.0,765.0,776.0
1,110.0,110.0,662.0,177.0,84.0,1989.0,135.0,55.0,426.0,206.0,...,522.0,38.0,734.0,1819.0,576.0,65.0,61.0,345.0,68.0,49.0
2,207.0,1672.0,1248.0,20421.0,318.0,3863.0,30727.0,5875.0,1786.0,7503.0,...,433.0,3172.0,1052.0,613.0,1890.0,1252.0,6110.0,173.0,2977.0,3152.0
3,178.0,245.0,60.0,1333.0,183.0,317.0,1344.0,190.0,153.0,136.0,...,103.0,5841.0,169.0,95.0,50.0,60.0,18310.0,34.0,105.0,2516.0
4,5.0,12.0,9.0,77.0,3.0,22.0,297.0,11.0,23.0,19.0,...,12.0,15.0,57.0,10.0,8.0,15.0,22.0,0.0,8.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35602,1048.0,1733.0,588.0,2920.0,678.0,2021.0,2216.0,1565.0,1047.0,866.0,...,704.0,501.0,1561.0,934.0,590.0,319.0,1232.0,313.0,664.0,688.0
35603,646.0,1396.0,1039.0,2184.0,500.0,3624.0,2414.0,3586.0,498.0,1609.0,...,1206.0,439.0,644.0,1828.0,923.0,633.0,1070.0,524.0,1043.0,509.0
35604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
